In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


# Data

In [2]:
df_train = pd.read_csv('../input/digit-recognizer/train.csv')
df_train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df_test = pd.read_csv('../input/digit-recognizer/test.csv')
df_test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df_y = df_train['label']
df_y.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [5]:
df_x = df_train.drop(['label'], axis = 1)
df_x.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
x_train = np.array(df_x)
np.shape(x_train)

(42000, 784)

In [7]:
y_train = np.array(df_y)
np.shape(y_train)

(42000,)

In [8]:
x_train = x_train / 255.0

In [9]:
test = np.array(df_test)
np.shape(test)

(28000, 784)

In [10]:
test = test / 255.0

In [11]:
x_train = x_train.reshape(-1, 28, 28, 1)
np.shape(x_train)

(42000, 28, 28, 1)

In [12]:
test = test.reshape(-1, 28, 28, 1)
np.shape(test)

(28000, 28, 28, 1)

In [13]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes = 10)
np.shape(y_train)

Using TensorFlow backend.


(42000, 10)

# Training

In [14]:
from keras import models, layers
from keras.callbacks import EarlyStopping

x = layers.Input((28, 28, 1))
h = layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1))(x)
h = layers.MaxPooling2D((2, 2))(h)
h = layers.Dropout(0.2)(h)
h = layers.Conv2D(64, (3, 3), activation = 'relu')(h)
h = layers.MaxPooling2D((2, 2))(h)
h = layers.Dropout(0.2)(h)
h = layers.Flatten()(h)
h = layers.Dense(128, activation = 'relu')(h)
h = layers.Dropout(0.2)(h)
output = layers.Dense(10, activation = 'softmax')(h)

model = models.Model(x, output)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
early_stopping = EarlyStopping(monitor='val_accuracy', mode = 'max', restore_best_weights = True, patience = 5)
model.fit(x_train, y_train, epochs = 1000, batch_size = 100, callbacks = [early_stopping], validation_split=0.2)

Train on 33600 samples, validate on 8400 samples
Epoch 1/1000
33600/33600 [==============================] - 18s 525us/step - loss: 0.3207 - accuracy: 0.8990 - val_loss: 0.0797 - val_accuracy: 0.9756
Epoch 2/1000
33600/33600 [==============================] - 17s 501us/step - loss: 0.0974 - accuracy: 0.9696 - val_loss: 0.0557 - val_accuracy: 0.9837
Epoch 3/1000
33600/33600 [==============================] - 16s 489us/step - loss: 0.0656 - accuracy: 0.9798 - val_loss: 0.0445 - val_accuracy: 0.9863
Epoch 4/1000
33600/33600 [==============================] - 17s 508us/step - loss: 0.0539 - accuracy: 0.9824 - val_loss: 0.0411 - val_accuracy: 0.9877
Epoch 5/1000
33600/33600 [==============================] - 17s 496us/step - loss: 0.0448 - accuracy: 0.9858 - val_loss: 0.0393 - val_accuracy: 0.9882
Epoch 6/1000
33600/33600 [==============================] - 17s 509us/step - loss: 0.0368 - accuracy: 0.9886 - val_loss: 0.0342 - val_accuracy: 0.9898
Epoch 7/1000
33600/33600 [===================

In [15]:
result = model.predict(test)

In [16]:
result = np.argmax(result, axis = 1)

In [17]:
submission = pd.DataFrame(result)
submission.head()

,0
0,2
1,0
2,9
3,9
4,3


In [18]:
submission.rename(columns = {0 : 'Label'}, inplace = True)
submission.head()

,Label
0,2
1,0
2,9
3,9
4,3


In [19]:
submission['ImageId'] = submission.index + 1
submission.head()

,Label,ImageId
0,2,1
1,0,2
2,9,3
3,9,4
4,3,5


In [20]:
submission = pd.DataFrame(submission, columns = ['ImageId','Label'])
submission.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [21]:
submission.to_csv('CNN_digit_recognizer.csv', index = False)